In [1]:
import urllib.request
import re
import pandas as pd
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup
#spectacleId="34525"
#spectacleId="36956"
spectacleId="20185"
# specify which URL/web page we are going to be scraping
url = "https://www.lesarchivesduspectacle.net/?IDX_Spectacle=" + spectacleId
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")
#the div where the information is
#the div where the information is
div_interested = soup.find('div', class_='fiche__footer') 
column_label=[]
column_value=[]
for row in div_interested.find_all('table')[0].findAll('tr'):
    th = row.findAll('th')[0].contents[0]
    td = row.findAll('td')[0].contents[0]
    column_label.append(th)
    column_value.append(td)
new_table = pd.DataFrame(columns=column_label, index= [0]) # I know the size
for row in div_interested.find_all('table')[0].findAll('tr'):
    th = row.findAll('th')[0].contents[0]
    td = row.findAll('td')[0].contents[0]
    new_table.at[0,th] = td
new_table["spectacleId"]=spectacleId
#Recuperer creation_date - organism_creation 
#the div where the information is
div_spectacle = soup.find('div', id='div_Spectacle')
p_spectacle = div_spectacle.find_all('p')[0]
if(p_spectacle):
    for time in p_spectacle.findAll('time'):
        if time.has_attr('datetime'):
            new_table["typeDate"]=time.previousSibling
            new_table["date"]=time['datetime']
    for organism in p_spectacle.findAll('a',class_ ='c_Organisme'):
        if organism.has_attr('href'):
            new_table["organismeId"]=organism['href']
            new_table["organismeName"]=organism.contents[0].strip()
new_table["to_analyse"]=p_spectacle.contents[0].strip()
new_table
#table[0] class="f-spectacle__equipe"    table[1]  class="f-spectacle__equipe"  
#table_production = div_spectacle.find_all('table')[2]
#print(table_production)


,Genre,Durée,Pays,spectacleId,typeDate,date,organismeId,organismeName,to_analyse
0,Danse – à partir de 7 ans\r\n,1 heure\r\n,France,20185,Création le,2009-11-03,?IDX_Organisme=837&Saison=2009,Maison de la Danse de Lyon,Création le


In [ ]:
#Question qui fait le plus de coproductions?
df_coproduction = pd.DataFrame(columns=['performanceId', 'isArsenalCopro', 'isCarreauCopro', 'isCCAMCopro','isGrandTheatreCopro' ])


In [26]:
#pour un spectacle voir les coproductions
#https://www.lesarchivesduspectacle.net/?IDX_Spectacle=90517#spectacle-partenaires-de-production
#Perchée dans les arbres 8 coproductions 

import urllib.request
import re
import pandas as pd
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup
#spectacleId="34525"
#spectacleId="36956"
spectacleId="90517"
# specify which URL/web page we are going to be scraping
url = "https://www.lesarchivesduspectacle.net/?IDX_Spectacle=" + spectacleId +"#spectacle-partenaires-de-production"
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")
#the div where the information is
#the div where the information is
table = soup.find('table', class_='f-spectacle__production-bis') 
# Down below we'll add our inmates to this list
inmates_list = []

# BeautifulSoup provides nice ways to access the data in the parsed
# page. Here, we'll use the select method and pass it a CSS style
# selector to grab all the rows in the table (the rows contain the 
# inmate names and ages).

for table_row in table.find_all('tr'):
    # Each tr (table row) has three td HTML elements (most people 
    # call these table cels) in it (first name, last name, and age)
    cells = table_row.findAll('td')
    # Our table has one exception -- a row without any cells.
    # Let's handle that special case here by making sure we
    # have more than zero cells before processing the cells
    if len(cells) > 0:
        organism = cells[1].text.strip()
        organism_element = cells[1].find('a',class_='c_Organisme b')
        organismId = organism_element['href'].replace("?IDX_Organisme=","")
        type_aide = cells[0].text.strip()
        if(type_aide != ''):
            type_aide_master = type_aide
        
        else:
            type_aide = type_aide_master
        inmate = {'performanceId': spectacleId,'type_aide': type_aide,'organism': organism,'organismId': organismId}
        inmates_list.append(inmate)

df = pd.DataFrame(inmates_list)
df

,performanceId,type_aide,organism,organismId
0,90517,Coproduction,L'Arsenal de Metz (Metz),439
1,90517,Coproduction,Ballet de Lorraine (Nancy),1798
2,90517,Coproduction,Le Carreau (Forbach),1914
3,90517,Coproduction,FRAC Lorraine (Metz),12533
4,90517,Coproduction,ICI - CCN Montpellier (Montpellier),40842
5,90517,Coproduction,La Maison CDCN (Uzès),50763
6,90517,Coproduction,Scènes croisées de Lozère (Mende),793
7,90517,Coproduction,Le Vent des Forêts (Fresnes-au-Mont),26625
8,90517,Aide,Région Grand Est (Strasbourg),44746
9,90517,Aide,DRAC Grand Est (Strasbourg),44731
